In [1]:
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils,loss_and_accuracy_curve_plotter,testing
from torchvision import transforms
from timeit import default_timer as timer 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Setup hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.00020703742384961855
CONFIG_NAME = 50


/home/h6x/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup directories

root_dir = "/home/h6x/git_projects/data_processing/processed_data/adjacency_pers_images_npy_county/experimet_3/npy_combined"
annotation_file_path ="/home/h6x/git_projects/data_processing/processed_data/adjacency_pers_images_npy_county/experimet_3/annotations_npy_2_classes_only_h0h1_class_inbalanced_exp_3.csv"

In [3]:
# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.ToTensor()
])

In [4]:
# Create DataLoaders with help from data_setup.py
train_dataloader, validation_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    annotation_file_path=annotation_file_path,
    root_dir=root_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

In [5]:
# Create model with help from model_builder.py
# model = model_builder.SEResNeXt(CONFIG_NAME).to(device)
model = model_builder.SEResNet(CONFIG_NAME).to(device)

In [6]:
# Compute class weights
all_labels = np.array([y for _, y in train_dataloader.dataset])

In [7]:
len(all_labels)

2058

In [8]:
unique_labels, label_counts = np.unique(all_labels, return_counts=True)
print(f"unique_labels: {unique_labels}")
print(f"label_counts: {label_counts}")

unique_labels: [0 1]
label_counts: [1597  461]


In [10]:
label_counts[0]

1597

In [11]:
label_counts[1]

461

In [12]:
N = len(all_labels)
K = 2

In [13]:
N/(K*label_counts[0])

0.6443331246086412

In [14]:
N/(K*label_counts[1])

2.232104121475054

In [15]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(all_labels), y=all_labels)

In [16]:
class_weights

array([0.64433312, 2.23210412])